# Derive terrain attributes from a Digital Elevation Model (DEM)

## Description
The purpose of this notebook is to compute terrain attributes from a Digital Elevation Model (DEM) for a specific area of interest. These terrain attributes provide important information about the topographic characteristics of the landscape and are particularly useful for studying wetlands. The notebook calculates various terrain indices such as aspect, curvature, Topographic Wetness Index (TWI), Terrain Profile Index (TPI), and hillshade. These indices offer insights into slope, orientation, shape, hydrology, water flow patterns, and other factors relevant to wetlands. By deriving these terrain attributes, researchers and analysts can better understand wetland dynamics, assess habitats, model ecosystems, and plan conservation strategies. The resulting terrain indices enhance wetland mapping, classification, and analysis, facilitating more accurate and detailed studies related to wetland ecosystems.

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Inastall and load packages" cell.

## Install and load Packages

### Optional - Install onetime

In [1]:
# !pip install richdem
# !pip install xarray-spatial
# !pip install focal_stats

In [2]:
%matplotlib inline

import os
import math
import datacube
import warnings
import rasterio
import rioxarray
import xarray as xr
import richdem as rd
import numpy as np
import geopandas as gpd
import rasterio as rio
from xrspatial import focal
import matplotlib.pyplot as plt
from xrspatial import hillshade
from xrspatial import convolution
from datacube.utils import geometry
from odc.dscache.tools import tiling
from rasterio.enums import Resampling
from deafrica_tools.plotting import display_map
from datacube.utils.geometry import BoundingBox, Geometry
from datashader.transfer_functions import shade
from datacube.utils.geometry import Geometry
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.areaofinterest import define_area
from scipy.ndimage import label, distance_transform_edt

import warnings

warnings.filterwarnings("ignore")

### Set up a dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.

In [3]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40847,Workers: 1
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:43035,Total threads: 4
Dashboard: /user/mpho.sadiki@digitalearthafrica.org/proxy/37119/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:44997,


### Initialize Datacube

In [4]:
dc = datacube.Datacube(app="DEM")

### Load vector and plot area of interest

In [5]:
# Specify a prefix to identify the area of interest in the saved outputs
# By assigning the desired prefix, you can easily identify the outputs associated with the specific area of interest.
prefix = "test"

# Method 1: Specify the latitude, longitude, and buffer
aoi = define_area(lat=10.338, lon=-1.055, buffer=0.1)

# Method 2: Use a polygon as a GeoJSON or Esri Shapefile. 
# aoi = define_area(vector_path='data/KZN.geojson')

#Create a geopolygon and geodataframe of the area of interest
geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)

# Get the latitude and longitude range of the geopolygon
lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

display_map(x=lon_range, y=lat_range)

### Explore the product names, resolution, and measurements

In [6]:
product_name = ['dem_cop_30', 'dem_srtm_deriv']
resolution = (-30, 30)
measurements = 'elevation'
dc_measurements = dc.list_measurements()
dc_measurements.loc[product_name].drop('flags_definition', axis=1)

name    dtype units   nodata aliases
product        measurement                                           
dem_cop_30     elevation    elevation  float32     1      NaN     NaN
dem_srtm_deriv mrvbf            mrvbf    int16     1 -32768.0     NaN
               mrrtf            mrrtf    int16     1 -32768.0     NaN
               slope            slope  float32     1  -9999.0     NaN

### Set up reusable  query object 

In [7]:
dask_chunks = {'x': 2500, 'y': 2500}

# set up daatcube query object
query = {
    'resolution': resolution,
    'output_crs': 'epsg:6933',
    "geopolygon": geopolygon,
    'dask_chunks': dask_chunks
}

### Derive terrain attributes

### Export Elevation, MrVBF, MrRTF
### Compute Slope, Aspect, Curvature, Topographic Wetness Index (TWI) and Topographic Postion Index (TPI)

#### Topographic Position 
Helps distinguish topographic features such as a hilltop, valley bottom, exposed ridge, flat plain, upper or lower slope. It is calculated by comparing the elevation of each pixel to its surrounding neighbours.

#### Topographic Wetness
A useful model to estimate where water will accumulate in an area with elevation differences. It is a function of slope and the upstream contributing area.


In [8]:
# Define the output directory
output_dir = os.path.join("data/terrain_attributes/", prefix)

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Load the dem 30 m product
ds_elev = dc.load(product="dem_cop_30", measurements='elevation',
                  **query).squeeze()
# Load the ds_deriv dataset
ds_deriv = dc.load(product="dem_srtm_deriv",
                   measurements=['mrvbf', 'mrrtf', 'slope'],
                   **query).squeeze()

In [9]:
# Extract elevation data from the loaded dataset
dem = ds_elev.elevation

def fill_depressions(dem):
    # Perform depression filling using some morphological operations.
    filled_dem = dem.copy()
    while True:
        marker = filled_dem.min()
        filled_dem = np.maximum(dem, marker)
        labels, num_features = label(dem > filled_dem)
        if num_features == 0:
            break
        dem = np.maximum(dem, filled_dem)
    return filled_dem

filled_dem = fill_depressions(dem)


In [10]:
elevation_path = os.path.join(output_dir, "elevation.tif")
filled_dem.rio.to_raster(elevation_path,
                                compress="deflate",
                                compress_opts=dict(zlevel=6))


with rio.open(elevation_path) as src:
    elevation_profile = src.profile

# Export mrrtf, and mrvbf 
mrrtf_path = os.path.join(output_dir, "mrrtf.tif")
ds_deriv.mrrtf.rio.to_raster(mrrtf_path, **elevation_profile)

mrvbf_path = os.path.join(output_dir, "mrvbf.tif")
ds_deriv.mrvbf.rio.to_raster(mrvbf_path, **elevation_profile)


# Define the list of scales you want to calculate attributes for
scales = [30, 100, 500]  

for scale in scales:
    scale_suffix = f"{scale}"

    if scale == 30:
        # No need to resample, use the original filled_dem
        resampled_elevation_DataArray = filled_dem
        resampled_elevation = rd.rdarray(resampled_elevation_DataArray, no_data=-9999)
    else:
        # Resample the filled_dem to the current scale using bilinear interpolation
        resampled_elevation_DataArray = filled_dem.rio.reproject(
            resolution=(scale, scale), resampling=Resampling.bilinear, dst_crs=filled_dem.rio.crs
        )
        resampled_elevation = rd.rdarray(resampled_elevation_DataArray, no_data=-9999)



    
    slope = rd.TerrainAttribute(resampled_elevation, attrib='slope_radians')
    aspect = rd.TerrainAttribute(resampled_elevation, attrib='aspect')
    curvature = rd.TerrainAttribute(resampled_elevation, attrib='curvature')
    profile_curvature = rd.TerrainAttribute(resampled_elevation, attrib='profile_curvature')
    planform_curvature = rd.TerrainAttribute(resampled_elevation, attrib='planform_curvature')

    
    # Compute TWI 
    accum_d8 = rd.FlowAccumulation(resampled_elevation, method='D8')
    twi = np.log(accum_d8 / (np.tan(slope) + 0.01))
    
    # Compute TPI
    cellsize_x, cellsize_y = convolution.calc_cellsize(resampled_elevation_DataArray)
    outer_radius = str(cellsize_x * 30) + "m"
    inner_radius = str(cellsize_x * 25) + "m"
    kernel = convolution.annulus_kernel(cellsize_x, cellsize_y, outer_radius, inner_radius)
    tpi = resampled_elevation_DataArray.squeeze() - focal.apply(resampled_elevation_DataArray.squeeze(), kernel)

    attributes = [
        (slope, f"slope_{scale_suffix}.tif"),
        (aspect, f"aspect_{scale_suffix}.tif"),
        (curvature, f"curvature_{scale_suffix}.tif"),
        (profile_curvature, f"profile_curvature_{scale_suffix}.tif"),
        (planform_curvature, f"planform_curvature_{scale_suffix}.tif"),
        (twi, f"twi_{scale_suffix}.tif"),
        (tpi, f"tpi_{scale_suffix}.tif")
    ]

    # Inside the loop where you save attribute GeoTIFFs
    for attribute, output_filename in attributes:
        output_path = os.path.join(output_dir, output_filename)

        # Create the output GeoTIFF file and write the attribute data
        with rio.open(output_path, 'w', **elevation_profile, quiet=True) as dst:
            dst.write(attribute, 1)
            

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 


A Slope calculation (radians)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.0217528====================== ] (99% - 0.0s - 1 threads)

A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.0351529====================== ] (99% - 0.0s - 1 threads)

A Curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.00699219===================== ] (99% - 0.0s - 1 threads)

A Profile curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.0142106====================== ] (99% - 0.0s - 1 threads)

A Planform curvature attribute calcu

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 


A Slope calculation (radians)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.00272785===================== ] (99% - 0.0s - 1 threads)

A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.00367978===================== ] (99% - 0.0s - 1 threads)

A Curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.00131666===================== ] (99% - 0.0s - 1 threads)

A Profile curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.00195444===================== ] (99% - 0.0s - 1 threads)

A Planform curvature attribute calcu

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top 


A Slope calculation (radians)
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.000546203==================== ] (98% - 0.0s - 1 threads)

A Aspect attribute calculation
C Horn, B.K.P., 1981. Hill shading and the reflectance map. Proceedings of the IEEE 69, 14–47. doi:10.1109/PROC.1981.11918

t Wall-time = 0.000597975==================== ] (98% - 0.0s - 1 threads)

A Curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.000519592==================== ] (98% - 0.0s - 1 threads)

A Profile curvature attribute calculation
C Zevenbergen, L.W., Thorne, C.R., 1987. Quantitative analysis of land surface topography. Earth surface processes and landforms 12, 47–56.

t Wall-time = 0.000512245==================== ] (98% - 0.0s - 1 threads)

A Planform curvature attribute calcu

## Cartographic Depth to Water
The function calculates the DTW Index, which estimates the depth of surface water in meters based on the topographic characteristics of the terrain and the cost function for least-cost path calculation.
1. Calculates the threshold (t) for the minimal flow initiation area (FIA) based on the input parameter fia. The threshold represents how much area needs to accumulate downward the slope to result in a channel with simulated surface water. This step is necessary to determine which areas are likely to have surface water.
    * 0.25: This means that for water to start flowing, it needs to accumulate in an area of at least 0.25 hectares (2500 square meters).
    * 1: In this case, water needs to accumulate in an area of at least 1 hectare (10,000 square meters) to start flowing.
    * 2: For water to flow, it needs to accumulate in an area of at least 2 hectares (20,000 square meters).
    * 4: Here, the required minimum area for water accumulation is 4 hectares (40,000 square meters).
    * 10: Water needs to accumulate in an area of at least 10 hectares (100,000 square meters) to start flowing.
    * 16: Finally, water needs to accumulate in an area of at least 16 hectares (160,000 square meters) for it to flow downhill.
    

2. Uses the calculated threshold to compare the flow accumulationto identify channels with surface water and sets them to 1 in the flowLines raster. Cells with flow accumulation below the threshold are marked as null. This step creates a binary raster layer where 1 represents channels with surface water, and null represents other areas.

3. Calculates the least-cost path of slope (cost) starting from the identified flow lines. This step helps determine the least costly path from each cell to the flow lines, considering the slope of the terrain. The cost raster will contain values representing the minimum height difference between each cell and the flow path.

4. Computes the Cartographic Depth-to-Water (DTW) Index in meters. The DTWxha1 raster is calculated by multiplying the cost raster by the spatial resolution of the DEM and dividing it by 100. This step converts the cost values (originally in percent) to represent the estimated depth of water in meters for each cell in the study area.



A value of zero in the depth to water raster means that the water table is at or very close to the ground surface. This suggests that the location is likely to be a water body or an area with high groundwater levels. As the depth values increase from zero to higher positive numbers, it indicates increasing depths below the ground surface, representing areas where the water table is deeper underground.

In [11]:
slope = ds_deriv.slope #Note slope is already in percent

# Set the FIA values you want to use
fia_values = [0.25,1, 4]

def calcDTW(fia, dem, slope):
    # Step 1: Calculate the threshold for minimal flow initiation (t)
    t = fia * 10000 / (30 * 30)  # flow accumulation is based on the number of cells
    
    # Step 2: Calculate flow accumulation and identify flow lines
    filled_dem_rd = rd.rdarray(filled_dem, no_data=-9999)
    accum_d8 = rd.FlowAccumulation(filled_dem_rd, method='D8')
    flow_lines = accum_d8 >= t

    # Step 3: Calculate the least-cost of slope in percentage starting from the flow line
    #The distance transform calculates, for each cell, the Euclidean distance to the nearest non-flowing cell. 
    #This essentially calculates the distance to the nearest flow line (channel) for each cell, considering the terrain's topology
    cost_distance = distance_transform_edt(flow_lines)
    
    # Step 4: Compute DTW in meters
    DTW_in_meters = cost_distance * slope / 100 #divide by 100 to convert from percentage to meters
    
    return DTW_in_meters

# Loop through each FIA value and calculate DTW
for fia in fia_values:
    dtw_result = calcDTW(fia, filled_dem, slope)
    
    # Save the DTW result to a GeoTIFF file
    output_filename = f"{output_dir}/DTW_FIA_{fia}_ha.tif"
    with rasterio.open(output_filename, 'w', driver='GTiff', height=dtw_result.shape[0],
                       width=dtw_result.shape[1], count=1, dtype=str(dtw_result.dtype),
                       crs=dem.geobox.crs, transform=dem.geobox.affine) as dst:
        dst.write(dtw_result, 1)


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A O'Callaghan (1984)/Marks (1984) D8/D4 Flow Accumulation
C O'Callaghan, J.F., Mark, D.M., 1984. The Extraction of Drainage Networks from Digital Elevation Data. Computer vision, graphics, and image processing 28, 323--344.

c topology = D8
[================================================= ] (99% - 0.0s - 1 threads)
A Generic Flow Accumulation Algorithm
p Creating dependencies array...

d Source cells found = 355124
p Calculating flow accumulation...
t Wall-time       = 0.0372089 s============== ] (99% - 0.0s - 1 threads)


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A O'Callaghan (1984)/Marks (1984) D8/D4 Flow Accumulation
C O'Callaghan, J.F., Mark, D.M., 1984. The Extraction of Drainage Networks from Digital Elevation Data. Computer vision, graphics, and image processing 28, 323--344.

c topology = D8
[================================================= ] (99% - 0.0s - 1 threads)
A Generic Flow Accumulation Algorithm
p Creating dependencies array...

d Source cells found = 355124
p Calculating flow accumulation...
t Wall-time       = 0.0396511 s============== ] (99% - 0.0s - 1 threads)


Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)



A O'Callaghan (1984)/Marks (1984) D8/D4 Flow Accumulation
C O'Callaghan, J.F., Mark, D.M., 1984. The Extraction of Drainage Networks from Digital Elevation Data. Computer vision, graphics, and image processing 28, 323--344.

c topology = D8
[================================================= ] (99% - 0.0s - 1 threads)
A Generic Flow Accumulation Algorithm
p Creating dependencies array...

d Source cells found = 355124
p Calculating flow accumulation...
t Wall-time       = 0.0334059 s============== ] (99% - 0.0s - 1 threads)


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [12]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2024-03-13'